In [5]:
import json
import math
import time
import pandas as pd
from simul import node, edge, port, AMHS

# 레이아웃 데이터 로드
with open("fab_oht_layout_updated.json") as f:
    layout_data = json.load(f)

nodes = [node(n['id'], [n['x'], n['y']]) for n in layout_data['nodes']]

edges = [
    edge(
        source=next(node for node in nodes if node.id == rail['from']),
        dest=next(node for node in nodes if node.id == rail['to']),
        length=math.sqrt(
            (next(node for node in nodes if node.id == rail['to']).coord[0] -
             next(node for node in nodes if node.id == rail['from']).coord[0])**2 +
            (next(node for node in nodes if node.id == rail['to']).coord[1] -
             next(node for node in nodes if node.id == rail['from']).coord[1])**2
        ),
        max_speed=1000 if rail.get('curve', 0) == 1 else 5000
    )
    for rail in layout_data['rails']
]

ports = [
    port(
        name=p['name'],
        from_node=p['from_node'],
        to_node=p['to_node'],
        from_dist=p['distance']
    )
    for p in layout_data['ports']
]

# 🔹 소켓 없이 시뮬만 도는 함수
def run_benchmark(num_ohts=200, max_time=2000, time_step=0.1):
    amhs = AMHS(nodes=nodes, edges=edges, ports=ports, num_OHTs=num_ohts, max_jobs=1000)


    start = time.time()
    current_time = 0
    count = 0
    while current_time <= max_time:
        if count % 100 == 0:
            amhs.generate_job()
        amhs.assign_jobs()
        
        for oht in amhs.OHTs:
            oht.move(time_step, current_time)
        
        amhs.update_edge_metrics(current_time, time_window=60)
        
        for oht in amhs.OHTs:
            oht.cal_pos(time_step)
        
        current_time += time_step
        count += 1
    end = time.time()


    return end - start

# 여러 설정값 테스트
num_ohts_list = [100, 200, 500]
max_time_list = [1000, 2000, 4000]
time_step = 0.1

results = []
for num_ohts in num_ohts_list:
    for max_time in max_time_list:
        elapsed = run_benchmark(num_ohts=num_ohts, max_time=max_time, time_step=time_step)
        results.append({
            "num_ohts": num_ohts,
            "max_time": max_time,
            "time_step": time_step,
            "elapsed_sec": round(elapsed, 2)
        })
        print(f"num_ohts={num_ohts}, max_time={max_time} → {elapsed:.2f} sec")

# 결과를 데이터프레임으로 정리
df = pd.DataFrame(results)
df


num_ohts=100, max_time=1000 → 82.60 sec
num_ohts=100, max_time=2000 → 167.93 sec
num_ohts=100, max_time=4000 → 335.72 sec
num_ohts=200, max_time=1000 → 125.79 sec
num_ohts=200, max_time=2000 → 257.60 sec
num_ohts=200, max_time=4000 → 512.15 sec
num_ohts=500, max_time=1000 → 134.96 sec
num_ohts=500, max_time=2000 → 240.59 sec
num_ohts=500, max_time=4000 → 592.55 sec


,num_ohts,max_time,time_step,elapsed_sec
0,100,1000,0.1,82.60
1,100,2000,0.1,167.93
2,100,4000,0.1,335.72
3,200,1000,0.1,125.79
4,200,2000,0.1,257.60
5,200,4000,0.1,512.15
6,500,1000,0.1,134.96
7,500,2000,0.1,240.59
8,500,4000,0.1,592.55


In [6]:
import json
import math
import time
import pandas as pd
from simul import node, edge, port, AMHS

# 레이아웃 데이터 로드
with open("fab_oht_layout_updated.json") as f:
    layout_data = json.load(f)

nodes = [node(n['id'], [n['x'], n['y']]) for n in layout_data['nodes']]

edges = [
    edge(
        source=next(node for node in nodes if node.id == rail['from']),
        dest=next(node for node in nodes if node.id == rail['to']),
        length=math.sqrt(
            (next(node for node in nodes if node.id == rail['to']).coord[0] -
             next(node for node in nodes if node.id == rail['from']).coord[0])**2 +
            (next(node for node in nodes if node.id == rail['to']).coord[1] -
             next(node for node in nodes if node.id == rail['from']).coord[1])**2
        ),
        max_speed=1000 if rail.get('curve', 0) == 1 else 5000
    )
    for rail in layout_data['rails']
]

ports = [
    port(
        name=p['name'],
        from_node=p['from_node'],
        to_node=p['to_node'],
        from_dist=p['distance']
    )
    for p in layout_data['ports']
]

# 🔹 소켓 없이 시뮬만 도는 함수
def run_benchmark(num_ohts=200, max_time=2000, time_step=0.1):
    amhs = AMHS(nodes=nodes, edges=edges, ports=ports, num_OHTs=num_ohts, max_jobs=1000)


    start = time.time()
    current_time = 0
    count = 0
    while current_time <= max_time:
        if count % 100 == 0:
            amhs.generate_job()
        amhs.assign_jobs()
        
        for oht in amhs.OHTs:
            oht.move(time_step, current_time)
        
        amhs.update_edge_metrics(current_time, time_window=60)
        
        for oht in amhs.OHTs:
            oht.cal_pos(time_step)
        
        current_time += time_step
        count += 1
    end = time.time()


    return end - start

# 여러 설정값 테스트
num_ohts_list = [100, 200, 300, 400, 500]
max_time_list = [1000, 2000, 3000, 4000, 5000]
time_step = 0.1

results = []
for num_ohts in num_ohts_list:
    for max_time in max_time_list:
        elapsed = run_benchmark(num_ohts=num_ohts, max_time=max_time, time_step=time_step)
        results.append({
            "num_ohts": num_ohts,
            "max_time": max_time,
            "time_step": time_step,
            "elapsed_sec": round(elapsed, 2)
        })
        print(f"num_ohts={num_ohts}, max_time={max_time} → {elapsed:.2f} sec")

# 결과를 데이터프레임으로 정리
df = pd.DataFrame(results)
df


num_ohts=100, max_time=1000 → 84.73 sec
num_ohts=100, max_time=2000 → 173.13 sec
num_ohts=100, max_time=3000 → 254.95 sec
num_ohts=100, max_time=4000 → 338.98 sec
num_ohts=100, max_time=5000 → 416.87 sec
num_ohts=200, max_time=1000 → 123.42 sec
num_ohts=200, max_time=2000 → 251.60 sec
num_ohts=200, max_time=3000 → 377.92 sec
num_ohts=200, max_time=4000 → 511.78 sec
num_ohts=200, max_time=5000 → 627.32 sec
num_ohts=300, max_time=1000 → 131.33 sec
num_ohts=300, max_time=2000 → 279.98 sec
num_ohts=300, max_time=3000 → 375.97 sec
num_ohts=300, max_time=4000 → 514.65 sec
num_ohts=300, max_time=5000 → 499.41 sec
num_ohts=400, max_time=1000 → 134.49 sec
num_ohts=400, max_time=2000 → 290.21 sec
num_ohts=400, max_time=3000 → 411.78 sec
num_ohts=400, max_time=4000 → 560.82 sec
num_ohts=400, max_time=5000 → 407.60 sec
num_ohts=500, max_time=1000 → 154.00 sec
num_ohts=500, max_time=2000 → 259.04 sec
num_ohts=500, max_time=3000 → 285.15 sec
num_ohts=500, max_time=4000 → 493.05 sec
num_ohts=500, max

,num_ohts,max_time,time_step,elapsed_sec
0,100,1000,0.1,84.73
1,100,2000,0.1,173.13
2,100,3000,0.1,254.95
3,100,4000,0.1,338.98
4,100,5000,0.1,416.87
5,200,1000,0.1,123.42
6,200,2000,0.1,251.60
7,200,3000,0.1,377.92
8,200,4000,0.1,511.78
9,200,5000,0.1,627.32


In [8]:
import os
import json
import math
import pandas as pd
from simul import node, edge, port, AMHS

# -------------------------------
# 레이아웃 데이터 로드
# -------------------------------
with open("fab_oht_layout_updated.json") as f:
    layout_data = json.load(f)

nodes = [node(n['id'], [n['x'], n['y']]) for n in layout_data['nodes']]

edges = [
    edge(
        source=next(node for node in nodes if node.id == rail['from']),
        dest=next(node for node in nodes if node.id == rail['to']),
        length=math.sqrt(
            (next(node for node in nodes if node.id == rail['to']).coord[0] -
             next(node for node in nodes if node.id == rail['from']).coord[0])**2 +
            (next(node for node in nodes if node.id == rail['to']).coord[1] -
             next(node for node in nodes if node.id == rail['from']).coord[1])**2
        ),
        max_speed=1000 if rail.get('curve', 0) == 1 else 5000
    )
    for rail in layout_data['rails']
]

ports = [
    port(
        name=p['name'],
        from_node=p['from_node'],
        to_node=p['to_node'],
        from_dist=p['distance']
    )
    for p in layout_data['ports']
]

# -------------------------------
# pivot 데이터 생성 함수
# -------------------------------
def generate_pivot_dataset(num_ohts=200, max_time=2000, time_step=0.1, interval=10, save_path=None):
    amhs = AMHS(nodes=nodes, edges=edges, ports=ports,
                num_OHTs=num_ohts, max_jobs=1000)

    current_time = 0
    count = 0
    last_logged = -interval
    logs = []

    while current_time <= max_time:
        if count % 100 == 0:
            amhs.generate_job()
        amhs.assign_jobs()

        for oht in amhs.OHTs:
            oht.move(time_step, current_time)

        amhs.update_edge_metrics(current_time, time_window=60)

        for oht in amhs.OHTs:
            oht.cal_pos(time_step)

        # interval마다 로깅
        if current_time - last_logged >= interval:
            for e in amhs.edges:
                logs.append({
                    "time": int(current_time),
                    "edge_id": e.id,
                    "avg_speed": e.avg_speed
                })
            last_logged = current_time

        current_time += time_step
        count += 1

    # pivot 변환
    df = pd.DataFrame(logs)
    pivot_df = df.pivot(index="time", columns="edge_id", values="avg_speed").reset_index()

    if save_path:
        pivot_df.to_csv(save_path, index=False)
        print(f"✅ Saved {save_path} with shape {pivot_df.shape}")

    return pivot_df


# -------------------------------
# 여러 조건 × 반복 실행
# -------------------------------
save_dir = "datasets_pivot"
os.makedirs(save_dir, exist_ok=True)

num_ohts_list = [200, 300, 400, 500]
max_time_list = [1000, 2000]
repeats = 10  # 각 세팅마다 50개 저장

for num_ohts in num_ohts_list:
    for max_time in max_time_list:
        for run_id in range(1, repeats + 1):
            fname = f"pivot_oht{num_ohts}_time{max_time}_run{run_id}.csv"
            fpath = os.path.join(save_dir, fname)
            generate_pivot_dataset(num_ohts=num_ohts,
                                   max_time=max_time,
                                   interval=10,
                                   save_path=fpath)


✅ Saved datasets_pivot/pivot_oht200_time1000_run1.csv with shape (100, 3425)
✅ Saved datasets_pivot/pivot_oht200_time1000_run2.csv with shape (100, 3425)
✅ Saved datasets_pivot/pivot_oht200_time1000_run3.csv with shape (100, 3425)
✅ Saved datasets_pivot/pivot_oht200_time1000_run4.csv with shape (100, 3425)
✅ Saved datasets_pivot/pivot_oht200_time1000_run5.csv with shape (100, 3425)
✅ Saved datasets_pivot/pivot_oht200_time1000_run6.csv with shape (100, 3425)
✅ Saved datasets_pivot/pivot_oht200_time1000_run7.csv with shape (100, 3425)
✅ Saved datasets_pivot/pivot_oht200_time1000_run8.csv with shape (100, 3425)
✅ Saved datasets_pivot/pivot_oht200_time1000_run9.csv with shape (100, 3425)
✅ Saved datasets_pivot/pivot_oht200_time1000_run10.csv with shape (100, 3425)
✅ Saved datasets_pivot/pivot_oht200_time2000_run1.csv with shape (199, 3425)
✅ Saved datasets_pivot/pivot_oht200_time2000_run2.csv with shape (199, 3425)
✅ Saved datasets_pivot/pivot_oht200_time2000_run3.csv with shape (199, 3425

In [12]:
import networkit as nk

def get_central_edges(amhs, top_k=10):
    bet = nk.centrality.EdgeBetweenness(amhs.graph)
    bet.run()
    scores = bet.scores()
    edge_list = list(amhs.graph.iterEdges())
    ranked = sorted(zip(edge_list, scores), key=lambda x: -x[1])
    return [(amhs.nodes[u].id, amhs.nodes[v].id) for (u, v, _), s in ranked[:top_k]]


In [14]:
import os
import random
import pandas as pd
import networkx as nx
from simul import node, edge, port, AMHS
import json, math
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# -------------------------------
# 레이아웃 데이터 로드
# -------------------------------
with open("fab_oht_layout_updated.json") as f:
    layout_data = json.load(f)

nodes = [node(n['id'], [n['x'], n['y']]) for n in layout_data['nodes']]
edges = [
    edge(
        source=next(node for node in nodes if node.id == rail['from']),
        dest=next(node for node in nodes if node.id == rail['to']),
        length=math.sqrt(
            (next(node for node in nodes if node.id == rail['to']).coord[0] -
             next(node for node in nodes if node.id == rail['from']).coord[0])**2 +
            (next(node for node in nodes if node.id == rail['to']).coord[1] -
             next(node for node in nodes if node.id == rail['from']).coord[1])**2
        ),
        max_speed=1000 if rail.get('curve', 0) == 1 else 5000
    )
    for rail in layout_data['rails']
]
ports = [
    port(
        name=p['name'],
        from_node=p['from_node'],
        to_node=p['to_node'],
        from_dist=p['distance']
    )
    for p in layout_data['ports']
]

# -------------------------------
# networkx로 edge betweenness 계산
# -------------------------------
def compute_edge_betweenness(edges):
    G = nx.DiGraph()
    for e in edges:
        G.add_edge(e.source, e.dest, weight=e.length)

    edge_bw = nx.edge_betweenness_centrality(G, weight="weight")
    ranked = sorted(edge_bw.items(), key=lambda x: -x[1])
    return [(src, dst) for (src, dst), _ in ranked]

# -------------------------------
# dynamic cut/restore 함수
# -------------------------------
def dynamic_cut_restore(amhs, central_edges, all_edges,
                        central_cut_prob=0.1, random_cut_prob=0.02, restore_prob=0.05):
    for (src, dst) in central_edges:
        if amhs.graph.hasEdge(amhs.node_id_map[src], amhs.node_id_map[dst]):
            if random.random() < central_cut_prob:
                amhs.modi_edge(src, dst, [], is_removed=True)
        else:
            if random.random() < restore_prob:
                amhs.modi_edge(src, dst, [], is_removed=False)

    for (src, dst) in random.sample(all_edges, k=max(1, len(all_edges)//20)):
        if amhs.graph.hasEdge(amhs.node_id_map[src], amhs.node_id_map[dst]):
            if random.random() < random_cut_prob:
                amhs.modi_edge(src, dst, [], is_removed=True)
        else:
            if random.random() < restore_prob:
                amhs.modi_edge(src, dst, [], is_removed=False)

# -------------------------------
# 시뮬레이션 (dynamic events 포함)
# -------------------------------
def simulate_with_dynamic_events(amhs, central_edges, all_edges,
                                 max_time=2000, time_step=0.1,
                                 interval=10, save_path=None):
    logs, current_time, count, last_logged = [], 0, 0, -interval

    while current_time <= max_time:
        if count % 100 == 0:
            amhs.generate_job()
        amhs.assign_jobs()

        for oht in amhs.OHTs:
            oht.move(time_step, current_time)
        amhs.update_edge_metrics(current_time, time_window=60)
        for oht in amhs.OHTs:
            oht.cal_pos(time_step)

        if count % 50 == 0:
            dynamic_cut_restore(amhs, central_edges, all_edges)

        if current_time - last_logged >= interval:
            for e in amhs.edges:
                logs.append({
                    "time": int(current_time),
                    "edge_id": e.id,
                    "avg_speed": e.avg_speed
                })
            last_logged = current_time

        current_time += time_step
        count += 1

    df = pd.DataFrame(logs)
    pivot_df = df.pivot(index="time", columns="edge_id", values="avg_speed").reset_index()

    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        pivot_df.to_csv(save_path, index=False)
        print(f"✅ Saved {save_path} with shape {pivot_df.shape}")

    return pivot_df

# -------------------------------
# 개별 작업 함수 (병렬 실행용)
# -------------------------------
def run_single_experiment(args):
    num_ohts, max_time, run_id, save_dir, central_edges, all_edges = args
    fname = f"dynamic_oht{num_ohts}_time{max_time}_run{run_id}.csv"
    fpath = os.path.join(save_dir, fname)

    amhs = AMHS(nodes=nodes, edges=edges, ports=ports,
                num_OHTs=num_ohts, max_jobs=1000)

    simulate_with_dynamic_events(amhs,
                                 central_edges=central_edges[:20],  # top 20
                                 all_edges=all_edges,
                                 max_time=max_time,
                                 time_step=0.1,
                                 interval=10,
                                 save_path=fpath)

# -------------------------------
# 여러 조건 × 반복 실행 (병렬)
# -------------------------------
save_dir = "datasets_dynamic"
os.makedirs(save_dir, exist_ok=True)

# central edges 한 번만 계산
edge_list = [(e.source, e.dest) for e in edges]
central_edges = compute_edge_betweenness(edges)
all_edges = edge_list

num_ohts_list = [200, 300, 400, 500]
max_time_list = [1000, 2000]
repeats = 10

tasks = []
for num_ohts in num_ohts_list:
    for max_time in max_time_list:
        for run_id in range(1, repeats + 1):
            tasks.append((num_ohts, max_time, run_id, save_dir, central_edges, all_edges))

# 병렬 실행 + 진행률 표시
with Pool(processes=min(cpu_count(), 8)) as pool:
    for _ in tqdm(pool.imap_unordered(run_single_experiment, tasks), total=len(tasks)):
        pass


Process SpawnPoolWorker-1:
Process SpawnPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Process SpawnPoolWorker-3:
  0%|          | 0/80 [00:00<?, ?it/s]Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/simul/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/anaconda3/envs/simul/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/anaconda3/envs/simul/lib/python3.12/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/opt/homebrew/anaconda3/envs/simul/lib/python3.12/multiprocessing/queues.py", line 389, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'run_single_experiment' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
  File "/opt/homebrew/anaconda3/envs/simul/lib/python3

KeyboardInterrupt: 

In [16]:
import json
import math
import time
import pandas as pd
from simul import node, edge, port, AMHS

# -------------------------------
# 레이아웃 데이터 로드
# -------------------------------
with open("fab_oht_layout_updated.json") as f:
    layout_data = json.load(f)

nodes = [node(n['id'], [n['x'], n['y']]) for n in layout_data['nodes']]
edges = [
    edge(
        source=next(node for node in nodes if node.id == rail['from']),
        dest=next(node for node in nodes if node.id == rail['to']),
        length=math.sqrt(
            (next(node for node in nodes if node.id == rail['to']).coord[0] -
             next(node for node in nodes if node.id == rail['from']).coord[0])**2 +
            (next(node for node in nodes if node.id == rail['to']).coord[1] -
             next(node for node in nodes if node.id == rail['from']).coord[1])**2
        ),
        max_speed=1000 if rail.get('curve', 0) == 1 else 5000
    )
    for rail in layout_data['rails']
]
ports = [
    port(
        name=p['name'],
        from_node=p['from_node'],
        to_node=p['to_node'],
        from_dist=p['distance']
    )
    for p in layout_data['ports']
]

# -------------------------------
# 시뮬레이션 실행 (소켓 없이)
# -------------------------------
def run_benchmark(num_ohts=200, max_time=2000, time_step=0.1):
    amhs = AMHS(nodes=nodes, edges=edges, ports=ports, num_OHTs=num_ohts, max_jobs=1000)

    start = time.time()
    current_time, count = 0, 0
    while current_time <= max_time:
        if count % 100 == 0:
            amhs.generate_job()
        amhs.assign_jobs()

        for oht in amhs.OHTs:
            oht.move(time_step, current_time)
        amhs.update_edge_metrics(current_time, time_window=60)
        for oht in amhs.OHTs:
            oht.cal_pos(time_step)

        current_time += time_step
        count += 1
    end = time.time()
    return end - start

# -------------------------------
# 반복 실행 + 평균
# -------------------------------
num_ohts_list = [100, 200, 300, 400, 500]
max_time_list = [1000, 2000, 3000, 4000, 5000]
time_step = 0.1
repeats = 5  # 각 설정을 5회 반복

results = []

for num_ohts in num_ohts_list:
    for max_time in max_time_list:
        elapsed_times = []
        for r in range(repeats):
            elapsed = run_benchmark(num_ohts=num_ohts, max_time=max_time, time_step=time_step)
            elapsed_times.append(elapsed)
            print(f"Run {r+1}/{repeats} → OHT={num_ohts}, Time={max_time}, Elapsed={elapsed:.2f}s")

        avg_elapsed = sum(elapsed_times) / len(elapsed_times)
        results.append({
            "num_ohts": num_ohts,
            "max_time": max_time,
            "time_step": time_step,
            "repeats": repeats,
            "avg_elapsed_sec": round(avg_elapsed, 2)
        })
        print(f"✅ AVG → OHT={num_ohts}, Time={max_time}, Avg={avg_elapsed:.2f}s")

# -------------------------------
# 결과 DataFrame
# -------------------------------
df = pd.DataFrame(results)
print(df)


Run 1/5 → OHT=100, Time=1000, Elapsed=85.27s


KeyboardInterrupt: 